# ASSIGNMENT #2
Niki Deeny
Due Friday 2016-07-08 at 11AM CST

## HW2.0:

1) How do you merge  two sorted  lists/arrays of records of the form [key, value]? Where is this  used in Hadoop MapReduce? [Hint within the shuffle]


In [991]:
mkdir Join

In [992]:
%%writefile Array1.dat
1|hello
Blabla|1,2,4,5
3.6|Hi

Overwriting Array1.dat


In [993]:
%%writefile Array2.dat
Woo|5,7,8
4|yaay
2|woot

Overwriting Array2.dat


In [994]:
%%writefile MRJoin.py

import sys, os, re
from mrjob.job import MRJob

class MRJoin(MRJob):

    SORT_VALUES = True

    def mapper(self, _, line):    
        splits = line.rstrip("\n").split("|")
        key = splits[0]
        yield key, splits

    def reducer(self, key, values):
        for value in values:
            yield key, value

if __name__ == '__main__':
    MRJoin.run()

Overwriting MRJoin.py


In [995]:
!python MRJoin.py Array1.dat Array2.dat

No configs found; falling back on auto-configuration
ignoring partitioner keyword arg (requires real Hadoop): 'org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner'
Creating temp directory /var/folders/r5/xnt6t6nd343fqlxyqjxqz3n50qpq8v/T/MRJoin.z084224.20160710.015241.328283
Running step 1 of 1...
Streaming final output from /var/folders/r5/xnt6t6nd343fqlxyqjxqz3n50qpq8v/T/MRJoin.z084224.20160710.015241.328283/output...
"1"	["1", "hello"]
"2"	["2", "woot"]
"3.6"	["3.6", "Hi"]
"4"	["4", "yaay"]
"Blabla"	["Blabla", "1,2,4,5"]
"Woo"	["Woo", "5,7,8"]
Removing temp directory /var/folders/r5/xnt6t6nd343fqlxyqjxqz3n50qpq8v/T/MRJoin.z084224.20160710.015241.328283...


2) What is  a combiner function in the context of Hadoop? 

    A combiner function is used as an intermediate step to summarize the outputs of the mapper before it goes to the reducer.

3) Give an example where it can be used and justify why it should be used in the context of this problem.

    It is used when you want to decrease the number of records going to the reducer by summarizing data further in each worker node.

4) What is the Hadoop shuffle?

    The Hadoop Shuffle is the process of moving map outputs to the reducers.

## HW2.1: Counters as a debugging aid (and for getting work done, but please use sparingly as they are heavy)

Consumer complaints dataset: Use Counters to do EDA (exploratory data analysis and to monitor progress)
Counters are lightweight objects in Hadoop that allow you to keep track of system progress in both the map and reduce stages of processing. By default, Hadoop defines a number of standard counters in "groups"; these show up in the jobtracker webapp, giving you information such as "Map input records", "Map output records", etc. 

While processing information/data using MapReduce job, it is a challenge to monitor the progress of parallel threads running across nodes of distributed clusters. Moreover, it is also complicated to distinguish between the data that has been processed and the data which is yet to be processed. The MapReduce Framework offers a provision of user-defined Counters, which can be effectively utilized to monitor the progress of data across nodes of distributed clusters.

Use the Consumer Complaints  Dataset provide here to complete this question:

     https://www.dropbox.com/s/vbalm3yva2rr86m/Consumer_Complaints.csv?dl=0

The consumer complaints dataset consists of diverse consumer complaints, which have been reported across the United States regarding various types of loans. The dataset consists of records of the form:

Complaint ID,Product,Sub-product,Issue,Sub-issue,State,ZIP code,Submitted via,Date received,Date sent to company,Company,Company response,Timely response?,Consumer disputed?

Here’s is the first few lines of the  of the Consumer Complaints  Dataset:

Complaint ID,Product,Sub-product,Issue,Sub-issue,State,ZIP code,Submitted via,Date received,Date sent to company,Company,Company response,Timely response?,Consumer disputed?
1114245,Debt collection,Medical,Disclosure verification of debt,Not given enough info to verify debt,FL,32219,Web,11/13/2014,11/13/2014,"Choice Recovery, Inc.",Closed with explanation,Yes,
1114488,Debt collection,Medical,Disclosure verification of debt,Right to dispute notice not received,TX,75006,Web,11/13/2014,11/13/2014,"Expert Global Solutions, Inc.",In progress,Yes,
1114255,Bank account or service,Checking account,Deposits and withdrawals,,NY,11102,Web,11/13/2014,11/13/2014,"FNIS (Fidelity National Information Services, Inc.)",In progress,Yes,
1115106,Debt collection,"Other (phone, health club, etc.)",Communication tactics,Frequent or repeated calls,GA,31721,Web,11/13/2014,11/13/2014,"Expert Global Solutions, Inc.",In progress,Yes,

#### User-defined Counters

Now, let’s use MapReduce Counters to identify the number of complaints pertaining to debt collection, mortgage and other categories (all other categories get lumped into this one) in the consumer complaints dataset. Basically produce the distribution of the Product column in this dataset using counters (limited to 3 counters here).

Hadoop offers Job Tracker, an UI tool to determine the status and statistics of all jobs. Using the job tracker UI, developers can view the Counters that have been created. Screenshot your  job tracker UI as your job completes and include it here. Make sure that your user defined counters are visible. 

In [996]:
!curl -L https://www.dropbox.com/s/vbalm3yva2rr86m/Consumer_Complaints.csv?dl=0 > Consumer_Complaints.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:--  0:00:02 --:--:--     0
100 48.5M  100 48.5M    0     0  1243k      0  0:00:39  0:00:39 --:--:-- 1426k


In [997]:
!head Consumer_Complaints.csv
!echo "-------------------"
!tail Consumer_Complaints.csv

Complaint ID,Product,Sub-product,Issue,Sub-issue,State,ZIP code,Submitted via,Date received,Date sent to company,Company,Company response,Timely response?,Consumer disputed?
1114245,Debt collection,Medical,Disclosure verification of debt,Not given enough info to verify debt,FL,32219,Web,11/13/2014,11/13/2014,"Choice Recovery, Inc.",Closed with explanation,Yes,
1114488,Debt collection,Medical,Disclosure verification of debt,Right to dispute notice not received,TX,75006,Web,11/13/2014,11/13/2014,"Expert Global Solutions, Inc.",In progress,Yes,
1114255,Bank account or service,Checking account,Deposits and withdrawals,,NY,11102,Web,11/13/2014,11/13/2014,"FNIS (Fidelity National Information Services, Inc.)",In progress,Yes,
1115106,Debt collection,"Other (phone, health club, etc.)",Communication tactics,Frequent or repeated calls,GA,31721,Web,11/13/2014,11/13/2014,"Expert Global Solutions, Inc.",In progress,Yes,
1115890,Credit reporting,,Incorrect information on credit report,Information is

In [998]:
#Remove first row of csv (column names)
!sed 1d Consumer_Complaints.csv > Consumer_Complaints_alt.csv

In [999]:
%%writefile Counter.py
from mrjob.job import MRJob

class Counter(MRJob):

    def mapper(self, _, line):
        line = line.strip().lower().split(",")
        product = line[1]
        if product in ["debt collection", "mortgage"]:
            self.increment_counter('product', product, 1)
        else:
            self.increment_counter('product', "other", 1)

if __name__ == '__main__':
    Counter().run()

Overwriting Counter.py


In [1000]:
!python Counter.py Consumer_Complaints_alt.csv

No configs found; falling back on auto-configuration
Creating temp directory /var/folders/r5/xnt6t6nd343fqlxyqjxqz3n50qpq8v/T/Counter.z084224.20160710.015348.752839
Running step 1 of 1...
Counters: 3
	product
		debt collection=44372
		mortgage=125752
		other=142788
Streaming final output from /var/folders/r5/xnt6t6nd343fqlxyqjxqz3n50qpq8v/T/Counter.z084224.20160710.015348.752839/output...
Removing temp directory /var/folders/r5/xnt6t6nd343fqlxyqjxqz3n50qpq8v/T/Counter.z084224.20160710.015348.752839...


## HW2.2: Analyze the performance of your Mappers, Combiners and Reducers using Counters



For this brief study the Input file will be one record (the next line only): 
foo foo quux labs foo bar quux


#### HW2.2 Part 1
Perform a word count analysis of this single record dataset using a Mapper and Reducer based WordCount (i.e., no combiners are used here) using user defined Counters to count up how many time the mapper and reducer are called. What is the value of your user defined Mapper Counter, and Reducer Counter after completing this word count job. The answer  should be 1 and 4 respectively. Please explain.


In [1001]:
%%writefile line.txt
foo foo quux labs foo bar quux

Overwriting line.txt


In [1002]:
%%writefile WordCount.py
from mrjob.job import MRJob
import sys

class MRWordFreqCount(MRJob):
    def mapper(self, _, line):
        # Counts the number of times mapper is called
        self.increment_counter('group', 'Num_mapper_calls', 1)
        for word in line.split(" "):
            yield word.lower(), 1

    def reducer(self, word, counts):
        # Counts the number of times reducer is called
        self.increment_counter('group', 'Num_reducer_calls', 1)
        yield word, sum(counts)

if __name__ == '__main__':
    MRWordFreqCount.run()

Overwriting WordCount.py


In [1003]:
from WordCount import MRWordFreqCount
mr_job = MRWordFreqCount(args=['line.txt'])

with mr_job.make_runner() as runner:
    runner.run()

    # Print the wordcount output
    for line in runner.stream_output():
        key, value = mr_job.parse_output_line(line)
        print key, value

# Print the count of mapper/reducer usage
print runner.counters()

bar 1
foo 3
labs 1
quux 2
[{'group': {'Num_mapper_calls': 1, 'Num_reducer_calls': 4}}]


You can see above that the word count has completed, and the number of times the mapper is called is 1, whereas the number of times the reducer is called is 4.

#### HW2.2 Part 2

Perform a word count analysis of the Issue column of the Consumer Complaints  Dataset using a Mapper and Reducer based WordCount (i.e., no combiners used anywhere)  using user defined Counters to count up how many times the mapper and reducer are called. What is the value of your user defined Mapper Counter, and Reducer Counter after completing your word count job. 


In [1004]:
%%writefile WordCount_CC.py
from mrjob.job import MRJob
import sys

class MRWordFreqCount_CC(MRJob):
    def mapper(self, _, line):
        # Counts the number of times mapper is called
        self.increment_counter('group', 'Num_mapper_calls', 1)
        line = line.strip().lower().split(",")
        issue = line[3]
        for word in issue.split(" "): # Counting words in "Issue"
            yield word.lower(), 1

    def reducer(self, word, counts):
        # Counts the number of times reducer is called
        self.increment_counter('group', 'Num_reducer_calls', 1)
        yield word, sum(counts)

if __name__ == '__main__':
    MRWordFreqCount_CC.run()

Overwriting WordCount_CC.py


In [1005]:
from WordCount_CC import MRWordFreqCount_CC
mr_job = MRWordFreqCount_CC(args=['Consumer_Complaints_alt.csv'])

with mr_job.make_runner() as runner:
    runner.run()

# Print the count of mapper/reducer usage
print runner.counters()

[{'group': {'Num_mapper_calls': 312912, 'Num_reducer_calls': 172}}]


Mapper was called 312,912 times, Reducer was called 172 times.

#### HW2.2 Part 3

Perform a word count analysis of the Issue column of the Consumer Complaints  Dataset using a Mapper, Reducer, and standalone combiner (i.e., not an in-memory combiner) based WordCount using user defined Counters to count up how many time the mapper, combiner, reducer are called. What is the value of your user defined Mapper Counter, combiner counter, and Reducer Counter after completing your word count job. 


In [1006]:
%%writefile WordCount_CC_Combiner.py
from mrjob.job import MRJob
import sys

class MRWordFreqCount_CC_Combiner(MRJob):
    
    def mapper(self, _, line):
        # Counts the number of times mapper is called
        self.increment_counter('group', 'Num_mapper_calls', 1)
        line = line.strip().lower().split(",")
        issue = line[3]
        for word in issue.split(" "): # Counting words in "Issue"
            yield word.lower(), 1
    
    def combiner(self, word, counts):
        self.increment_counter('group','Num_combiner_calls',1)
        yield word, sum(counts)

    def reducer(self, word, counts):
        # Counts the number of times reducer is called
        self.increment_counter('group', 'Num_reducer_calls', 1)
        yield word, sum(counts)

if __name__ == '__main__':
    MRWordFreqCount_CC_Combiner.run()

Overwriting WordCount_CC_Combiner.py


In [1007]:
from WordCount_CC_Combiner import MRWordFreqCount_CC_Combiner
mr_job = MRWordFreqCount_CC_Combiner(args=['Consumer_Complaints_alt.csv'])

with mr_job.make_runner() as runner:
    runner.run()

# Print the count of mapper/reducer usage
print runner.counters()

[{'group': {'Num_combiner_calls': 319, 'Num_mapper_calls': 312912, 'Num_reducer_calls': 172}}]


Combiner was called 319 times.  Mapper calls and Reducer calls were unchanged.

### HW2.2.1:  

Using a single reducer perform a sort of the words in decreasing order of word counts. Present the top 50 terms and their frequency. If there are ties please sort the tokens in alphanumeric/string order. Present bottom 10 tokens (least frequent items). 

HINT: You will need a second MRStep for the sort part. Step 1 will be the usual word count, while step 2 will be a sort step. Please use the Hadoop/MRJob framework to perform the sort. Please do NOT use any of the built-in sorts  from  python.

In [1008]:
%%writefile HW221_1.py
from mrjob.job import MRJob
import sys

class MRWordFreqCount_CC_Combiner(MRJob):
    
    def jobconf(self):
        orig_jobconf = super(MRWordFreqCount_CC_Combiner, self).jobconf()        
        custom_jobconf = {
            'mapred.output.key.comparator.class': 'org.apache.hadoop.mapred.lib.KeyFieldBasedComparator',
            'mapred.text.key.comparator.options': '-k1rn',
            'mapred.reduce.tasks': '1',
        }
        combined_jobconf = orig_jobconf
        combined_jobconf.update(custom_jobconf)
        self.jobconf = combined_jobconf
        return combined_jobconf
    
    def mapper(self, _, line):
        line = line.strip().lower().split(",")
        issue = line[3]
        for word in issue.split(" "): # Counting words in "Issue"
            yield word.lower(), 1
    
    def combiner(self, word, counts):
        yield word, sum(counts)

    def reducer(self, word, counts):
        yield word, sum(counts)

if __name__ == '__main__':
    MRWordFreqCount_CC_Combiner.run()

Overwriting HW221_1.py


In [1009]:
from HW221_1 import MRWordFreqCount_CC_Combiner
mr_job = MRWordFreqCount_CC_Combiner(args=['Consumer_Complaints_alt.csv'])
wc={}

with mr_job.make_runner() as runner:
    runner.run()

    for line in runner.stream_output():
        key,value =  mr_job.parse_output_line(line)
        wc[key] = value

    with open('WordCount_Output.txt', 'w') as f:
            for k in wc.keys():
                f.writelines( k + "," + str(wc[k]) +"\n")

In [1010]:
%%writefile HW221_Top50.py
from mrjob.job import MRJob
import sys

class HW221(MRJob):
    #Sort
    SORT_VALUES = True
    
    def mapper(self, _, line):
        line = line.strip().lower().split(",")
        word, count = line
        yield "1", (count, word)

    def reducer(self, key, values):
        thelist = []
        order = 0
        
        for count, words in values:
            order += 1
            thelist.append((int(count),words,order))
            thelist.sort(reverse=True) #Sort biggest to smallest
        
        # Show top 50
        for k in thelist:
            if k[2] >= int(order)-50:
                print k[1],k[0]

if __name__ == '__main__':
    HW221.run()


Overwriting HW221_Top50.py


In [1011]:
!python HW221_Top50.py --jobconf mapred.reduce.tasks=1 WordCount_Output.txt

No configs found; falling back on auto-configuration
ignoring partitioner keyword arg (requires real Hadoop): 'org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner'
Creating temp directory /var/folders/r5/xnt6t6nd343fqlxyqjxqz3n50qpq8v/T/HW221_Top50.z084224.20160710.015533.040734
Running step 1 of 1...
modification 70487
credit 50894
problems 9484
"application 8625
to 8401
unable 8178
billing 8158
other 7886
disputes 6938
tactics 6920
communication 6920
reporting 6559
lease 6337
the 6248
low 5663
funds 5663
caused 5663
by 5663
being 5663
process 5505
verification 5214
disclosure 5214
managing 5006
investigation 4858
company's 4858
identity 4729
account 4476
card 4405
charged 976
for 929
i 925
didn't 925
dispute 904
fees 807
expect 807
shopping 672
unsolicited 640
issuance 640
transfer 597
balance 597
scam 566
issues 538
amount 98
payment 92
credited 92
convenience 75
checks 75
day 71
amt 71
incorrect/missing 64
disclosures 64
Streaming final output from /var/folders/r5/xnt6t6nd343fqlx

In [1012]:
%%writefile HW221_Bottom10.py
from mrjob.job import MRJob
import sys

class HW221(MRJob):
    #Sort
    SORT_VALUES = True
    
    def mapper(self, _, line):
        line = line.strip().lower().split(",")
        word, count = line
        yield "1", (count, word)

    def reducer(self, key, values):
        thelist = []
        order = 0
        
        for count, words in values:
            order += 1
            thelist.append((int(count),words,order))
            thelist.sort() #Sort smallest to largest
        
        # Show Bottom 10
        for k in thelist:
            if k[2] >= int(order)-10:
                print k[1],k[0]

if __name__ == '__main__':
    HW221.run()


Overwriting HW221_Bottom10.py


In [1013]:
!python HW221_Bottom10.py --jobconf mapred.reduce.tasks=1 WordCount_Output.txt

No configs found; falling back on auto-configuration
ignoring partitioner keyword arg (requires real Hadoop): 'org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner'
Creating temp directory /var/folders/r5/xnt6t6nd343fqlxyqjxqz3n50qpq8v/T/HW221_Bottom10.z084224.20160710.015540.144066
Running step 1 of 1...
credited 92
payment 92
amount 98
dispute 904
didn't 925
i 925
for 929
charged 976
to 8401
"application 8625
problems 9484
Streaming final output from /var/folders/r5/xnt6t6nd343fqlxyqjxqz3n50qpq8v/T/HW221_Bottom10.z084224.20160710.015540.144066/output...
Removing temp directory /var/folders/r5/xnt6t6nd343fqlxyqjxqz3n50qpq8v/T/HW221_Bottom10.z084224.20160710.015540.144066...


### HW2.2.2:   
Repeat HW2.2.1 using 3 reducers. Use the same code as in HW2.2.1  with just one modification 
to the command line: just add --jobconf mapred.reduce.tasks=3 as see presented here: 

    python HW2.2WordCount.py --jobconf mapred.reduce.tasks=3 oneLinerTextFile.txt

Describe what you see. Is this correct?

In [1014]:
%%writefile HW222.py
from mrjob.job import MRJob
import sys

class MRWordFreqCount_CC_Combiner(MRJob):
    
    def jobconf(self):
        orig_jobconf = super(MRWordFreqCount_CC_Combiner, self).jobconf()        
        custom_jobconf = {
            'mapred.output.key.comparator.class': 'org.apache.hadoop.mapred.lib.KeyFieldBasedComparator',
            'mapred.text.key.comparator.options': '-k1rn',
            'mapred.reduce.tasks': '3', # change to 3 reducers
        }
        combined_jobconf = orig_jobconf
        combined_jobconf.update(custom_jobconf)
        self.jobconf = combined_jobconf
        return combined_jobconf
    
    def mapper(self, _, line):
        line = line.strip().lower().split(",")
        issue = line[3]
        for word in issue.split(" "): # Counting words in "Issue"
            yield word.lower(), 1
    
    def combiner(self, word, counts):
        yield word, sum(counts)

    def reducer(self, word, counts):
        yield word, sum(counts)

if __name__ == '__main__':
    MRWordFreqCount_CC_Combiner.run()

Overwriting HW222.py


In [1015]:
from HW222 import MRWordFreqCount_CC_Combiner
mr_job = MRWordFreqCount_CC_Combiner(args=['Consumer_Complaints_alt.csv'])
wc={}

with mr_job.make_runner() as runner:
    runner.run()

    for line in runner.stream_output():
        key,value =  mr_job.parse_output_line(line)
        wc[key] = value

    with open('WordCount_Output_222.txt', 'w') as f:
            for k in wc.keys():
                f.writelines( k + "," + str(wc[k]) +"\n")

In [1016]:
!python HW221_Top50.py --jobconf mapred.reduce.tasks=3 WordCount_Output_222.txt

No configs found; falling back on auto-configuration
ignoring partitioner keyword arg (requires real Hadoop): 'org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner'
Creating temp directory /var/folders/r5/xnt6t6nd343fqlxyqjxqz3n50qpq8v/T/HW221_Top50.z084224.20160710.015558.357419
Running step 1 of 1...
modification 70487
credit 50894
problems 9484
"application 8625
to 8401
unable 8178
billing 8158
other 7886
disputes 6938
tactics 6920
communication 6920
reporting 6559
lease 6337
the 6248
low 5663
funds 5663
caused 5663
by 5663
being 5663
process 5505
verification 5214
disclosure 5214
managing 5006
investigation 4858
company's 4858
identity 4729
account 4476
card 4405
charged 976
for 929
i 925
didn't 925
dispute 904
fees 807
expect 807
shopping 672
unsolicited 640
issuance 640
transfer 597
balance 597
scam 566
issues 538
amount 98
payment 92
credited 92
convenience 75
checks 75
day 71
amt 71
incorrect/missing 64
disclosures 64
Streaming final output from /var/folders/r5/xnt6t6nd343fqlx

In [1017]:
!python HW221_Bottom10.py --jobconf mapred.reduce.tasks=3 WordCount_Output.txt

No configs found; falling back on auto-configuration
ignoring partitioner keyword arg (requires real Hadoop): 'org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner'
Creating temp directory /var/folders/r5/xnt6t6nd343fqlxyqjxqz3n50qpq8v/T/HW221_Bottom10.z084224.20160710.015602.249641
Running step 1 of 1...
credited 92
payment 92
amount 98
dispute 904
didn't 925
i 925
for 929
charged 976
to 8401
"application 8625
problems 9484
Streaming final output from /var/folders/r5/xnt6t6nd343fqlxyqjxqz3n50qpq8v/T/HW221_Bottom10.z084224.20160710.015602.249641/output...
Removing temp directory /var/folders/r5/xnt6t6nd343fqlxyqjxqz3n50qpq8v/T/HW221_Bottom10.z084224.20160710.015602.249641...


#### Conclusion
Seems to still be the same output

## HW2.3: Shopping Cart Analysis
Product Recommendations: The action or practice of selling additional products or services 
to existing customers is called cross-selling. Giving product recommendation is 
one of the examples of cross-selling that are frequently used by online retailers. 
One simple method to give product recommendations is to recommend products that are frequently
browsed together by the customers.
  
For this homework use the online browsing behavior dataset located at: 

       https://www.dropbox.com/s/zlfyiwa70poqg74/ProductPurchaseData.txt?dl=0

Each line in this dataset represents a browsing session of a customer. 
On each line, each string of 8 characters represents the id of an item browsed during that session. 
The items are separated by spaces.

Here are the first few lines of the ProductPurchaseData 
FRO11987 ELE17451 ELE89019 SNA90258 GRO99222 
GRO99222 GRO12298 FRO12685 ELE91550 SNA11465 ELE26917 ELE52966 FRO90334 SNA30755 ELE17451 FRO84225 SNA80192 
ELE17451 GRO73461 DAI22896 SNA99873 FRO86643 
ELE17451 ELE37798 FRO86643 GRO56989 ELE23393 SNA11465 
ELE17451 SNA69641 FRO86643 FRO78087 SNA11465 GRO39357 ELE28573 ELE11375 DAI54444 

In [1018]:
!curl -L https://www.dropbox.com/s/zlfyiwa70poqg74/ProductPurchaseData.txt?dl=0 > ProductPurchaseData.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 3377k  100 3377k    0     0   579k      0  0:00:05  0:00:05 --:--:--  986k


In [1019]:
!head ProductPurchaseData.txt

FRO11987 ELE17451 ELE89019 SNA90258 GRO99222 
GRO99222 GRO12298 FRO12685 ELE91550 SNA11465 ELE26917 ELE52966 FRO90334 SNA30755 ELE17451 FRO84225 SNA80192 
ELE17451 GRO73461 DAI22896 SNA99873 FRO86643 
ELE17451 ELE37798 FRO86643 GRO56989 ELE23393 SNA11465 
ELE17451 SNA69641 FRO86643 FRO78087 SNA11465 GRO39357 ELE28573 ELE11375 DAI54444 
ELE17451 GRO73461 DAI22896 SNA99873 FRO18919 DAI50921 SNA80192 GRO75578 
ELE17451 ELE59935 FRO18919 ELE23393 SNA80192 SNA85662 SNA91554 DAI22177 
ELE17451 SNA69641 FRO18919 SNA90258 ELE28573 ELE11375 DAI14125 FRO78087 
ELE17451 GRO73461 DAI22896 SNA80192 SNA85662 SNA90258 DAI46755 FRO81176 ELE66810 DAI49199 DAI91535 GRO94758 ELE94711 DAI22177 
ELE17451 SNA69641 DAI91535 GRO94758 GRO99222 FRO76833 FRO81176 SNA80192 DAI54690 ELE37798 GRO56989 


Do some exploratory data analysis of this dataset guided by the following questions:. 

How many unique items are available from this supplier?

In [1021]:
with open("ProductPurchaseData.txt", "rb") as f:
    data=f.read().strip().lower().split(" \n")
    line = [k.split(" ") for k in data if k]

allitems = []
for k in line:
    for i in k:
        allitems.append(i)

# Total unique number of items
len(set(allitems))

12592

Using a single reducer: Report your findings such as number of unique products; largest basket; report the top 50 most frequently purchased items,  using Hadoop Map-Reduce. 

#### Largest Basket

In [1022]:
%%writefile HW23_LargestBakset.py
from mrjob.job import MRJob
import sys

class LargestBasket(MRJob):
    #Sort
    SORT_VALUES = True

    def mapper(self, _, lines):
        line=lines.strip().lower().split()
        basket=len(line)
        values = []
        values.append(basket)
        values.append(line)
        yield "key", values

    def reducer(self, key, values):
        thelist = []
        order = 0

        for basket, line in values:
            order += 1
            thelist.append((int(basket),line,order))
            thelist.sort(reverse=True) #Sort biggest to smallest
        
        print "The Largest Basket has %s items in it. Here are the contents: %s"  % (thelist[0][0],thelist[0][1])

if __name__ == '__main__':
    LargestBasket.run()

Overwriting HW23_LargestBakset.py


In [990]:
!python HW23_LargestBakset.py ProductPurchaseData.txt

No configs found; falling back on auto-configuration
ignoring partitioner keyword arg (requires real Hadoop): 'org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner'
Creating temp directory /var/folders/r5/xnt6t6nd343fqlxyqjxqz3n50qpq8v/T/HW23_LargestBakset.z084224.20160710.014734.069831
Running step 1 of 1...
The Largest Basket has 37 items in it. Here are the contents: ['gro21487', 'fro85978', 'dai89320', 'sna53220', 'sna55762', 'gro46854', 'ele38511', 'sna66583', 'fro79579', 'fro92469', 'fro40251', 'gro97448', 'dai35347', 'fro31317', 'fro87622', 'sna42518', 'ele53126', 'ele17451', 'gro32086', 'ele30327', 'dai58206', 'dai38969', 'ele16038', 'dai75645', 'dai55148', 'gro94173', 'ele43952', 'fro69613', 'gro81647', 'gro73461', 'fro24098', 'ele96667', 'gro88324', 'gro82670', 'gro12815', 'sna37475', 'ele24369']
Streaming final output from /var/folders/r5/xnt6t6nd343fqlxyqjxqz3n50qpq8v/T/HW23_LargestBakset.z084224.20160710.014734.069831/output...
Removing temp directory /var/folders/r5/xnt6

#### Top 50 Most Frequently Purhcased Items

In [889]:
%%writefile HW23_WC.py
from mrjob.job import MRJob
import sys

class MRWordFreqCount_CC_Combiner(MRJob):
    
    def mapper(self, _, lines):
        line=lines.strip().lower().split()
        for word in line: # Counting words in "Issue"
            yield word.lower(), 1
    
    def combiner(self, word, counts):
        yield word, sum(counts)

    def reducer(self, word, counts):
        # Counts the number of times reducer is called
        yield word, sum(counts)

if __name__ == '__main__':
    MRWordFreqCount_CC_Combiner.run()

Overwriting HW23_1.py


In [893]:
# !python WordCount_CC_Combiner.py --jobconf mapred.reduce.tasks=1 Consumer_Complaints_alt.csv

from HW23_WC import MRWordFreqCount_CC_Combiner
mr_job = MRWordFreqCount_CC_Combiner(args=['ProductPurchaseData.txt'])
wc={}

with mr_job.make_runner() as runner:
    runner.run()

    for line in runner.stream_output():
        key,value =  mr_job.parse_output_line(line)
        wc[key] = value

    with open('PP_Output.txt', 'w') as f:
            for k in wc.keys():
                f.writelines( k + "," + str(wc[k]) +"\n")

In [899]:
!python HW221_Top50.py --jobconf mapred.reduce.tasks=1 PP_Output.txt

No configs found; falling back on auto-configuration
ignoring partitioner keyword arg (requires real Hadoop): 'org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner'
Creating temp directory /var/folders/r5/xnt6t6nd343fqlxyqjxqz3n50qpq8v/T/HW221_Top50.z084224.20160710.011836.942594
Running step 1 of 1...
gro81087 965
dai55148 945
dai91290 925
gro24246 922
gro64900 917
sna18183 99
gro38636 99
fro61250 99
sna97370 98
sna87482 98
sna63339 98
fro49726 98
fro47796 98
gro94871 97
gro90585 97
gro66628 97
ele93905 97
ele73625 97
ele67905 97
ele40371 97
fro73386 96
sna18094 95
gro92215 95
ele31596 95
ele24694 95
dai51778 95
sna79613 94
sna30908 94
gro54782 94
gro16765 94
fro91992 94
fro73019 94
sna90161 93
sna37363 93
sna28607 93
dai71083 93
dai46921 93
sna79852 92
sna48838 92
gro73939 92
ele76310 92
ele24467 92
ele12808 92
dai50189 92
dai20585 92
dai20027 92
sna53983 91
sna35089 91
sna12350 91
gro88085 91
fro56023 91
Streaming final output from /var/folders/r5/xnt6t6nd343fqlxyqjxqz3n50qpq8v/T/H

### 2.3.1 OPTIONAL 
Using 2 reducers:  Report your findings such as number of unique products; largest basket; report the top 50 most frequently purchased items,  their frequency,  and their relative frequency (break ties by sorting the products alphabetical order) etc. using Hadoop Map-Reduce. 

## HW2.4 (Computationally prohibitive but then again Hadoop can handle this) Pairs

From a data mining perspective (and the aPriori algorihtm in particular), Support and Confidence are defined as follows:

SUPPORT
In data mining, the support value of X (where X is a collection of cooccurring items sometimes referred to as an item-set. E.g., a basket or subset of a basket) with respect to T  (a transaction database where each row is a transaction such as a basket of items that have been purchased)  is defined as the proportion of transactions in the  database which contains  the item-set X. (a relative frequency of sorts) 

CONFIDENCE 
The confidence value of a rule, X ==>  Y (where X is a collection of cooccurring items and Y is generally a single item. E.g., If Diapers and Beer then Cigars were also purchased), with respect to a set of transactions T, is the proportion of the transactions that contains X which also contains Y. (Think of it as  tgePr(Y|X) )

The pairs/stripes algorithm returns cooccurrence information that can be used directly to  calculate the confidence and support. Note that confidence for pair X ==>  Y will  differ from the relative frequency that results from stripes when X occurs by itself in transactions.


Suppose we want to recommend new products to the customer based on the products they
have already browsed on the online website. Write a map-reduce program 
to find products which are frequently browsed together. Fix the support count (cooccurence count) to s = 100 
(i.e. product pairs need to occur together at least 100 times to be considered frequent) 
and find pairs of items (sometimes referred to itemsets of size 2 in association rule mining) that have a support count of 100 or more.

List the top 50 product pairs with corresponding support count (aka frequency), and relative frequency or support (number of records where they coccur, the number of records where they coccur/the number of baskets in the dataset)  in decreasing order of support  for frequent (100>count) itemsets of size 2. 

Use the Pairs pattern  to  extract these frequent itemsets of size 2. Free free to use combiners if they bring value. Instrument your code with counters for count the number of times your mapper, combiner and reducers are called.  

Please output records of the following form for the top 50 pairs (itemsets of size 2): 

      item1, item2, support count, support  (OPTIONAL Feel free to add in confidence level also)



Fix the ordering of the pairs lexicographically (left to right), 
and break ties in support (between pairs, if any exist) 
by taking the first ones in lexicographically increasing order. 

Report  the compute time for the Pairs job. Describe the computational setup used (E.g., single computer; dual core; linux, number of mappers, number of reducers)
Instrument your mapper, combiner, and reducer to count how many times each is called using Counters and report these counts.


## HW2.5: Stripes
Repeat 2.4 using the stripes design pattern for finding cooccuring pairs (and out.

Report  the compute times for stripes job versus the Pairs job. Describe the computational setup used (E.g., single computer; dual core; linux, number of mappers, number of reducers)

Instrument your mapper, combiner, and reducer to count how many times each is called using Counters and report these counts. Discuss the differences in these counts between the Pairs and Stripes jobs